# FIT5196 Assessment 1 Task 1
#### Student Name:Balaji Ippagunta
#### Student ID: 29876451

Date: 15/04/2017

Version: 2.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* re
* json 



## 1. Introduction
This task comprises the web scraping of the Monash courses html file which has details of 400 units. The task is to scrape the data from the html file and pasrse the data in json and xml formats for the upstream systems.
The required tasks are the following:
More details for each task will be given in the following sections.

## 2.  Import libraries 

In [1]:
import re
import json

## 3. Regular Expressions 
Regular expressions required for extracting data from the file

For segmenting the units we are extracting everything between `"<div class=\"hbk-banner-box\">"` and `<!-- /\.content-inner__main --></div>`

In [2]:
segmentRegexStr=(r"<div class=\"hbk-banner-box\">\n"
	r"((.*|\n)*)\n"
	r"<!-- /\.content-inner__main --></div>")
unitcodeRegexStr = r"<h1 class=\"[a-z]+_[a-z]+\"><span class=\"unitcode\">(.*)<\/span> - (.*)<span class=\"hbk-archive-only\"> - \d+<\/span><\/h1>"
synopsisRegexStr = (r"<h2 class=\"hbk-heading\">Synopsis<\/h2>\n"
	r"<div>\n"
	r"((?:.|\n)*)\n"
	r"<\/div>\n"
	r"<h2 class=\"hbk-heading\">Outcomes</h2>")
prereqRegexStr = (r"<p class=\"hbk-preamble-heading\">Prerequisites</p>\n"
	r"<p>(.*)</p>")
sevenLengthAlphaNumericRegexStr=(r"([A-Z0-9]{7})")
corequisitesRegexStr=(r"<p class=\"hbk-preamble-heading\">Co-requisites</p>\n"
	r"<p>(.*)</p>")
prohibitionsRegexStr = (r"<p class=\"hbk-preamble-heading\">Prohibitions</p>\n"
	r"<p>(.*)</p>")
requirementsRegexStr = (r"<h2 class=\"hbk-heading\">Assessment</h2>\n"
	r"<div>((.?|\n)*)\n"
	r"</div>\n"
	r"<h2 class=")
everythingBetweenPRegexStr = (r"<p>(.*)</p>")
outcomeRegexStr=(r"<h2 class=\"hbk-heading\">Outcomes</h2>\n"
	r"<div>\n"
	r"((.?|\n)*)\n"
	r"</div>\n"
	r"<h2.*</h2>\n")
everythingBetweenLiRegexStr = r"<li>(.*?)</li>"
ceRegStr=(r"<p class=\"hbk-highlight-heading\">Chief examiner\(s\)</p>\n"
	r"((.?|\n)*)\n"
	r"<p")
everythingBetweenARegexStr=r"<a .*>(.*)</a>"

synopsisRegexComp = re.compile(synopsisRegexStr, re.MULTILINE)
unitcodeRegexComp = re.compile(unitcodeRegexStr, re.MULTILINE)
segmentRegexComp=re.compile(segmentRegexStr, re.MULTILINE)
prereqRegexComp = re.compile(prereqRegexStr, re.MULTILINE)
corequisitesRegexComp = re.compile(corequisitesRegexStr, re.MULTILINE)
sevenLengthAlphaNumericRegexComp = re.compile(sevenLengthAlphaNumericRegexStr, re.MULTILINE)
prohibitionsRegexComp= re.compile(prohibitionsRegexStr, re.MULTILINE)
requirementsRegexComp = re.compile(requirementsRegexStr,re.MULTILINE)
everythingBetweenPRegexComp = re.compile(everythingBetweenPRegexStr,re.MULTILINE)
outcomeRegexComp = re.compile(outcomeRegexStr,re.MULTILINE)
everythingBetweenLiRegexComp = re.compile(everythingBetweenLiRegexStr,re.MULTILINE)
ceRegComp = re.compile(ceRegStr,re.MULTILINE)
everythingBetweenARegexComp=re.compile(everythingBetweenARegexStr,re.MULTILINE)

## 4. Segmenting the file per unit 
Reading the file into the program and adding each unit's html data into a list as an element for further analysis.

In [3]:
readFile = open("29876451.txt").read()
segementedUnits=[]
for match in segmentRegexComp.findall(readFile):
   
    segementedUnits.append(match[0])

## 5. Extracting the data 

In [4]:
unitdictList=[]
for match in segementedUnits:
    
    #Extracting the Unit Id and Unit title
    unitCodeRaw=unitcodeRegexComp.findall(match)
    unitid=unitCodeRaw[0][0] #Unit ID
    if len(unitid)>7:
        unitid=unitid[1:8] #If unit ID is greater than 7 letters considering the last 7 letters
    unitname=unitCodeRaw[0][1] #Unit Title  
    
    #Extracting Synopsis
    synopsisMatch=synopsisRegexComp.findall(match)
    if len(synopsisMatch)>0:
        synopsisRaw=everythingBetweenPRegexComp.findall(synopsisMatch[0])
    synopsis=[]
    for ele in synopsisRaw:
        ele=re.sub('<span .*?.html\">', '', ele)
        ele=re.sub('</a></span>', '', ele)
        synopsis.append(ele)
    #If there is no synopsis then we are setting it as NA
    if len(synopsis)==0:
        synopsis="NA"
    else:
        synopsistemp=synopsis
        synopsis=[]
        #Removing specail characters to save them into JSON
        for ele in synopsistemp:
            ele=ele.replace("&amp;","&")
            ele=ele.replace("&gt;",">")
            ele=ele.replace("&lt;","<")
            synopsis.append(ele)
        synopsis=synopsis[0]
    #Extracting Prerequsite information
    prerqRaw=prereqRegexComp.findall(match)
    if len(prerqRaw)>0:
        prerequisitesList=sevenLengthAlphaNumericRegexComp.findall(prerqRaw[0])
    else:
        prerequisitesList=[]
    prerequisitesListtSet=list(dict.fromkeys(prerequisitesList))#Removing duplicate values
    #Extracting Corequsite information
    corequisitesMatch=corequisitesRegexComp.findall(match)
    if len(corequisitesMatch)>0:
        corequisitesList=sevenLengthAlphaNumericRegexComp.findall(corequisitesMatch[0])
    else:
        corequisitesList=[]
    corequisitesListSet=list(dict.fromkeys(corequisitesList))#Removing duplicate values
    #Adding the corequisites into prerequsites
    if len(corequisitesListSet)>0:
        prerequisitesListtSet=prerequisitesListtSet+corequisitesListSet
    #If there are no prerequsites then we are saving the value as NA    
    if len(prerequisitesListtSet)==0:
        prerequisitesListtSet="NA"
    elif len(prerequisitesListtSet)==1:
        prerequisitesListtSet=prerequisitesListtSet[0]
        
    if prerequisitesListtSet=="NA":
        pre_requistic_holder=prerequisitesListtSet
    else:
        pre_requistic_holder={"pre_requistic":prerequisitesListtSet}
    
    #Extracting Prohibitions information
    prohibitMatch=prohibitionsRegexComp.findall(match)
    if len(prohibitMatch)>0:
        prohibitionsList=sevenLengthAlphaNumericRegexComp.findall(prohibitMatch[0])
    else:
        prohibitionsList=[]
    prohibitionsSet=list(dict.fromkeys(prohibitionsList))#Removing duplicate values
    
    #If there are no prohibitions then we are saving the value as NA 
    if len(prohibitionsSet)==0:
        prohibitionsSet="NA"
    elif len(prohibitionsSet)==1:
        prohibitionsSet=prohibitionsSet[0]
    if (prohibitionsSet is not "NA") :
        probsetholder={"prohibision":prohibitionsSet}
    else :
        probsetholder=prohibitionsSet
    
    #Extracting Requirements information
    requirementsRaw=requirementsRegexComp.findall(match)

    if len(requirementsRaw)>0:
        requirementsListUnfiltered=everythingBetweenPRegexComp.findall(requirementsRaw[0][0])
    else:
        requirementsListUnfiltered=[]    
    requirementsList=[]
    #Removing unwanted tags and information from the requirements
    for ele in requirementsListUnfiltered:
       
        ele=re.sub('<a .*.html>', ' ', ele)
        ele=re.sub('<a .*print-url\">', ' ', ele)
        ele=re.sub('</a><span class=\"hbk-print-url\">', '', ele)
        ele=re.sub('\(<a .*</a>\)</span>', ' ', ele)
        ele=ele.replace("&amp;","&")
        ele=ele.replace("&gt;",">")
        ele=ele.replace("&lt;","<")
        requirementsList.append(ele)
    #If there are no requirements we are saving them as NA
    if len(requirementsList)==0:
        requirementsList="NA"
    elif len(requirementsList)==1:
        requirementsList=requirementsList[0]
    if (requirementsList is not "NA") :
        reqholder={"requirement":requirementsList}
    else :
        reqholder=requirementsList
    
    #Extracting Prohibitions information
    outcomeMatch=outcomeRegexComp.findall(match)
    outcomeList=[]
    if len(outcomeMatch)>0:
        outcomeList=everythingBetweenLiRegexComp.findall(outcomeMatch[0][0].replace('\n', ' '))
    if len(outcomeList)==1:
        outcomeList=outcomeList[0]    
    
    #Extracting Cheif Examiner information
    ceMatch=ceRegComp.findall(match)

    if(len(ceMatch)>0):
        cheifexaminer=everythingBetweenARegexComp.findall(ceMatch[0][0])
    else:
         cheifexaminer=[]
    #If there are no chief examiners we are saving them as TBA        
    if len(cheifexaminer)==0:
        ceholder="TBA"
    elif (len(cheifexaminer))==1:
        ceholder={"chief_examiner":cheifexaminer[0]}
    else:
        ceholder={"chief_examiner":cheifexaminer}    
    #saving all the attributes into a dictionary as key value pairs
    unitdict={"@id":unitid,"title":unitname,"synopsis":synopsis,"pre_requistics":pre_requistic_holder,"prohibisions":probsetholder,"requirements":reqholder,"outcomes":{"outcome":outcomeList},"chief_examiners":ceholder}
    #saving the dictionary into the list to create xml and json
    unitdictList.append(unitdict)

## 6 Creating JSON file

In [5]:
finalList=[]
for ele in unitdictList:
    dumpedeles = json.dumps(ele)
    loadedele = json.loads(dumpedeles)
    finalList.append(loadedele)
pre={"units":{"unit":finalList}}
finaljson= json.dumps(pre)
#Writing it into a file
jsonfile = open("JSONFile.json", "a")
jsonfile.write(""+finaljson)
jsonfile.close()
print("JSONFile.json file created")

JSONFile.json file created


## 7 Creating xml file

In [6]:
xmlList=""
xmlList=xmlList+"<?xml version=\"1.0\" encoding=\"UTF-8\" ?>"
xmlList=xmlList+"<units>"
for ele in unitdictList:
    xmlSubEle=""
    xmlSubEle=xmlSubEle+"<unit id='"+ele["@id"]+"'>"
    xmlSubEle=xmlSubEle+"<title>" +ele["title"]+"</title>"
    xmlSubEle=xmlSubEle+"<synopsis>" +ele["synopsis"]+"</synopsis>"
    xmlSubEle=xmlSubEle+"<pre_requistics>"
    xmlsubEleSub=""
    if ele["pre_requistics"] == "NA":
        xmlsubEleSub=ele["pre_requistics"]
    else:
        if isinstance(ele["pre_requistics"]["pre_requistic"], str):
            xmlsubEleSub=xmlsubEleSub+"<pre_requistic>"+ele["pre_requistics"]["pre_requistic"] +"</pre_requistic>"
        else:
            
            for subele in ele["pre_requistics"]["pre_requistic"]:
                xmlsubEleSub=xmlsubEleSub+"<pre_requistic>"+subele +"</pre_requistic>"
    xmlSubEle=xmlSubEle+xmlsubEleSub
    xmlSubEle=xmlSubEle+"</pre_requistics>"
    
    xmlSubEle=xmlSubEle+"<prohibisions>"
    xmlsubEleSub=""
    if ele["prohibisions"] == "NA":
        xmlsubEleSub=ele["prohibisions"]
    else:
        if isinstance(ele["prohibisions"]["prohibision"], str):
            xmlsubEleSub=xmlsubEleSub+"<prohibision>"+ele["prohibisions"]["prohibision"] +"</prohibision>"
        else:
            for subele in ele["prohibisions"]["prohibision"]:
                xmlsubEleSub=xmlsubEleSub+"<prohibision>"+subele +"</prohibision>"
    xmlSubEle=xmlSubEle+xmlsubEleSub
    xmlSubEle=xmlSubEle+"</prohibisions>"
    
    xmlSubEle=xmlSubEle+"<requirements>"
    xmlsubEleSub=""
    if ele["requirements"] == "NA":
        xmlsubEleSub=ele["requirements"]
    else:
        if isinstance(ele["requirements"]["requirement"], str):
            xmlsubEleSub=xmlsubEleSub+"<requirement>"+ele["requirements"]["requirement"] +"</requirement>"
        else:
            for subele in ele["requirements"]["requirement"]:
                xmlsubEleSub=xmlsubEleSub+"<requirement>"+subele +"</requirement>"
    xmlSubEle=xmlSubEle+xmlsubEleSub
    xmlSubEle=xmlSubEle+"</requirements>"
    
    xmlSubEle=xmlSubEle+"<outcomes>"
    xmlsubEleSub=""
    if isinstance(ele["outcomes"]["outcome"], str):
        ele["outcomes"]["outcome"]=ele["outcomes"]["outcome"].replace("<", " ").replace(">", " ")
        xmlsubEleSub=xmlsubEleSub+"<outcome>"+ele["outcomes"]["outcome"] +"</outcome>"
    else:
        for subele in ele["outcomes"]["outcome"]:
            subele=subele.replace("<", " ").replace(">", " ")
            xmlsubEleSub=xmlsubEleSub+"<outcome>"+subele +"</outcome>"
    xmlSubEle=xmlSubEle+xmlsubEleSub
    xmlSubEle=xmlSubEle+"</outcomes>"
    
    xmlSubEle=xmlSubEle+"<chief_examiners>"
    xmlsubEleSub=""
    if ele["chief_examiners"] == "TBA":
        xmlsubEleSub=ele["chief_examiners"]
    else:
        if isinstance(ele["chief_examiners"]["chief_examiner"], str):
            xmlsubEleSub=xmlsubEleSub+"<chief_examiner>"+ele["chief_examiners"]["chief_examiner"] +"</chief_examiner>"
        else:
            for subele in ele["chief_examiners"]["chief_examiner"]:
                xmlsubEleSub=xmlsubEleSub+"<chief_examiner>"+subele +"</chief_examiner>"
    xmlSubEle=xmlSubEle+xmlsubEleSub
    xmlSubEle=xmlSubEle+"</chief_examiners>"
    
    xmlSubEle=xmlSubEle+"</unit>"
    xmlList=xmlList+xmlSubEle
xmlList=xmlList+"</units>"
xmlList=xmlList.replace("&","&amp;")
xmlfile = open("XMLFile.xml", "a")
xmlfile.write(""+xmlList)
xmlfile.close()
print("XMLFile.xml file created")

XMLFile.xml file created
